In [18]:
from __future__ import print_function
import os
import numpy as np
from sklearn import preprocessing
np.random.seed(1337)
from sklearn.preprocessing import OneHotEncoder

import keras
from keras.utils.np_utils import to_categorical
from keras.layers import Dense, Input, Flatten
from keras.layers import Dropout, LSTM, Activation, Bidirectional
from keras.models import Model,Sequential
from keras.layers import Convolution1D, MaxPooling1D
import sys

import random

window = 20
firstFile = True

In [19]:
import glob, os
os.chdir("/home/amoro/punctuation/data")
for file in glob.glob("*.fea"):
    f = open(file)

    data_raw = []
    data = []

    for line in f:
        data_raw=line.split("\t")
        data.append(data_raw)
    
    #Padding

    null = [0,0,0,0,0,0,0,0,0,0,0,0,0,0, "sil\n"]

    for i in range(window-1):
        data = np.vstack([null, data])
    
    #Z normalize

    X = data[:, 0]
    X = preprocessing.scale(X.astype(float))
        

    #Assemble CNN data
    rnn_input = np.zeros((len(X),window,1))
    rnn_output = np.zeros((len(X),1)) 

    for i in range(len(X)-window):
    
        for j in range(window):
            rnn_input[i, j, :] = X[i+j]
            
        rnn_output[i] =  X[i+j+1]
        
    #Append to other files' data      
    if (firstFile):
        x_data = rnn_input
        y_data = rnn_output
        firstFile = False
    else:
        x_data = np.vstack([x_data, rnn_input])
        y_data = np.vstack([y_data, rnn_output])
      

    
#Shuffle data

indices = np.arange(x_data.shape[0])
np.random.shuffle(indices)

x_data = x_data[indices]
y_data = y_data[indices]

#Divide into train, test and validation data

len_data = len(x_data)

nb_test = int(len_data*0.15)
nb_validation = int(len_data*0.15)
nb_train = int(len_data*0.7)

end_valid = nb_train+nb_validation

x_train = x_data[0:nb_train]
y_train = y_data[0:nb_train]

x_valid = x_data[nb_train:end_valid]
y_valid = y_data[nb_train:end_valid]

x_test = x_data[end_valid:]
y_test = y_data[end_valid:]

In [20]:
model = Sequential()

model.add(LSTM(50, input_dim=1, input_length=window, return_sequences=True))
model.add(Dropout(0.5))

model.add(LSTM(30))
model.add(Dropout(0.25))

model.add(Dense(output_dim=1, activation='linear'))

/home/amoro/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:3: UserWarning: The `input_dim` and `input_length` arguments in recurrent layers are deprecated. Use `input_shape` instead.
  This is separate from the ipykernel package so we can avoid doing imports until
/home/amoro/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:3: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(50, return_sequences=True, input_shape=(20, 1))`
  This is separate from the ipykernel package so we can avoid doing imports until
/home/amoro/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:9: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=1, activation="linear")`
  if __name__ == '__main__':


In [21]:
earlyStopping=keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, verbose=0, mode='auto')

model.compile(loss='mse',optimizer='adam',metrics=['acc'])

model.fit(x_train, y_train, nb_epoch=30, shuffle=True, callbacks=[earlyStopping], 
          validation_data = (x_valid, y_valid), batch_size=1000)

Train on 94621 samples, validate on 20275 samples
Epoch 1/30
94621/94621 [==============================] - 100s - loss: 0.2025 - acc: 0.0076 - val_loss: 0.0826 - val_acc: 0.0087
Epoch 2/30
94621/94621 [==============================] - 100s - loss: 0.0784 - acc: 0.0076 - val_loss: 0.0404 - val_acc: 0.0087
Epoch 3/30
94621/94621 [==============================] - 106s - loss: 0.0509 - acc: 0.0076 - val_loss: 0.0274 - val_acc: 0.0087
Epoch 4/30
50000/94621 [==============>...............] - ETA: 52s - loss: 0.0435 - acc: 0.0080

KeyboardInterrupt: 